In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 4.9 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=b5cecb7348ce3633e6e69dd65fa1790a625e06ac4307c837dd0a47e04d2a6e06
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 2.3 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf
 教授资格证身份证.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529-a0611_UK		       dataframe_result_csv
 a0529-a0611_US		       dataframe_result_csv_EU
 a0530.csv		       dataframe_result_csv_UK
 a0531.csv		       Github
 a0601.csv		       heatmap529_UK.ipynb
 a0602.csv		       heatmap530.ipynb
 a0603.csv		       Heat_Map_Online_language.ipynb
 ccc_compiled.csv	       __MACOSX
 csv			       mapping_UK
 csv.zip		       mapping_US
 dataframe_csv		       non_violant1.csv
 dataframe_csv_EU	       non_violant2.csv
 dataframe_csv_UK	       UK
 dataframe_result_529_UK.csv   violant.csv
 dataframe_result_530_UK.csv  '新建 Microsoft PowerPoint 演示文稿.pptx'
 dataframe_result_531_UK.csv


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct_path_601 = 'csv/US06_06.csv'

In [8]:
ct = pd.read_csv(ct_path_601)

In [9]:
ct.shape[1]

47

In [10]:
columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [11]:
#len(columns1)
len(columns2)

47

In [12]:
ct.columns=columns2

In [13]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,19729044948_10151472828499949,Facebook,2020-06-06 20:16:26,2021-07-28 15:31:24,photo,NaN,NaN,"Yesterdays webinar Gender-Justice was, hopeful...",Protestors gather along 16th Street NW near th...,1,https://www.facebook.com/933thebeat/photos/a.3...,https://www.facebook.com/19729044948/posts/101...,56103,-6.000000,1.0,3,1,0,0,0,0,0,0,0,0,3,3,2,3,2,4,7,0,0,6,38774,93.3 The Beat Jamz,933thebeat,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Jacksonville's #1 Station for Hip-Hop and R&B ...,True,2009-01-13 16:46:07,en,0,38774|10151472828499949
1,193425634038780_2897594533621863,Facebook,2020-06-06 20:16:26,2021-06-15 21:27:28,link,George Floyd GoFundMe Campaign Has Received Mo...,incrediblestories.me,A GoFundMe page set up for George Floyd has re...,George Floyd GoFundMe Campaign Has Received Mo...,1,https://incrediblestories.me/george-floyd-gofu...,https://www.facebook.com/193425634038780/posts...,528349,3.857143,1.0,457,100,270,258,20,2,0,0,0,51,146,24,39,54,7,8,5,4,0,8,369803,Terry Bаms,TerryBams,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Army Vet | Best Selling Author | Father | Husb...,True,2011-06-14 05:05:53,en,0,369803|2897594533621863
2,52150999700_10158798739704701,Facebook,2020-06-06 20:16:22,2021-09-02 05:24:41,link,Watch 'Sesame Street' and CNN's town hall on r...,ew.com,'Sesame Street' and CNN presented a one-hour s...,Sesame Street and CNN presented a one-hour spe...,1,http://share.ew.com/X8cbJCW,https://www.facebook.com/52150999700/posts/101...,4609669,1.461864,1.0,628,174,32,188,3,6,1,3,0,36,477,35,52,81,15,19,23,6,0,11,15356,Entertainment Weekly,entertainmentweekly,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Entertainment Weekly brings the fun. With our...,True,2009-02-23 20:21:13,en,0,15356|10158798739704701
3,49727872493_10159055015087494,Facebook,2020-06-06 20:16:03,2021-09-03 11:28:55,link,Protesters pour into DC for city's largest dem...,fox5dc.com,Protesters pour into DC for city's largest dem...,"Here's the scene in downtown DC, where up to 2...",1,https://www.fox5dc.com/news/protesters-pour-in...,https://www.facebook.com/49727872493/posts/101...,1230384,6.021978,1.0,360,108,421,139,32,11,4,21,0,16,45,27,44,7,16,9,18,16,0,5,48870,Fox 5 DC,fox5dc,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Welcome to the official Facebook page for FOX ...,True,2008-12-16 17:46:26,en,0,48870|10159055015087494
4,50614167032_10158759715837033,Facebook,2020-06-06 20:15:57,2021-08-31 04:12:00,link,Outspoken Hoke County Sheriff Sounds Off on Ra...,spectrumlocalnews.com,Peterkin talks about what needs to be done.,Spectrum News anchor Tim Boyum chats with Hoke...,2,http://specne.ws/oHwpnh,https://www.facebook.com/50614167032/posts/101...,134561,-3.666667,1.0,4,1,0,4,0,0,0,0,0,0,5,3,4,3,2,3,7,6,0,3,400156,Spectrum News 1 North Carolina,SpectrumNews1NC,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Spectrum News 1 is a 24-hour local news channe...,True,2008-10-03 00:36:16,en,0,400156|10158759715837033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11360,57427307078_10159378232012079,Facebook

In [14]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [15]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [16]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [17]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [19]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [20]:
ct['NER-msg'] = ct['message'].apply(ner)

In [21]:
ct['NER-desc'] = ct['description'].apply(ner)

In [22]:
ct['NER-title'] = ct['title'].apply(ner)

In [23]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,"{'Washington': 1, 'D.C.': 1, 'Minneapolis': 1}",{}
1,{},{}
2,{},{}
3,{'DC': 1},{'DC': 1}
4,"{'Hoke County': 1, 'Raeford': 1}",{}


In [24]:
ct.to_csv('a0606.csv')

In [25]:
ct=pd.read_csv('a0606.csv')

In [26]:
!ls

 a0529-a0611_UK		       dataframe_result_531_UK.csv
 a0529-a0611_US		       dataframe_result_csv
 a0530.csv		       dataframe_result_csv_EU
 a0531.csv		       dataframe_result_csv_UK
 a0601.csv		       Github
 a0602.csv		       heatmap529_UK.ipynb
 a0603.csv		       heatmap530.ipynb
 a0606.csv		       Heat_Map_Online_language.ipynb
 a0607.csv		       __MACOSX
 ccc_compiled.csv	       mapping_UK
 csv			       mapping_US
 csv.zip		       non_violant1.csv
 dataframe_csv		       non_violant2.csv
 dataframe_csv_EU	       UK
 dataframe_csv_UK	       violant.csv
 dataframe_result_529_UK.csv  '新建 Microsoft PowerPoint 演示文稿.pptx'
 dataframe_result_530_UK.csv


In [27]:
###
#ct=pd.read_csv('a0601.csv')

In [28]:
ct

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title
0,0,19729044948_10151472828499949,Facebook,2020-06-06 20:16:26,2021-07-28 15:31:24,photo,NaN,NaN,"Yesterdays webinar Gender-Justice was, hopeful...",Protestors gather along 16th Street NW near th...,1,https://www.facebook.com/933thebeat/photos/a.3...,https://www.facebook.com/19729044948/posts/101...,56103,-6.000000,1.0,3,1,0,0,0,0,0,0,0,0,3,3,2,3,2,4,7,0,0,6,38774,93.3 The Beat Jamz,933thebeat,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Jacksonville's #1 Station for Hip-Hop and R&B ...,True,2009-01-13 16:46:07,en,0,38774|10151472828499949,"{'Washington': 1, 'D.C.': 1, 'Minneapolis': 1}",{},{}
1,1,193425634038780_2897594533621863,Facebook,2020-06-06 20:16:26,2021-06-15 21:27:28,link,George Floyd GoFundMe Campaign Has Received Mo...,incrediblestories.me,A GoFundMe page set up for George Floyd has re...,George Floyd GoFundMe Campaign Has Received Mo...,1,https://incrediblestories.me/george-floyd-gofu...,https://www.facebook.com/193425634038780/posts...,528349,3.857143,1.0,457,100,270,258,20,2,0,0,0,51,146,24,39,54,7,8,5,4,0,8,369803,Terry Bаms,TerryBams,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Army Vet | Best Selling Author | Father | Husb...,True,2011-06-14 05:05:53,en,0,369803|2897594533621863,{},{},{}
2,2,52150999700_10158798739704701,Facebook,2020-06-06 20:16:22,2021-09-02 05:24:41,link,Watch 'Sesame Street' and CNN's town hall on r...,ew.com,'Sesame Street' and CNN presented a one-hour s...,Sesame Street and CNN presented a one-hour spe...,1,http://share.ew.com/X8cbJCW,https://www.facebook.com/52150999700/posts/101...,4609669,1.461864,1.0,628,174,32,188,3,6,1,3,0,36,477,35,52,81,15,19,23,6,0,11,15356,Entertainment Weekly,entertainmentweekly,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Entertainment Weekly brings the fun. With our...,True,2009-02-23 20:21:13,en,0,15356|10158798739704701,{},{},{}
3,3,49727872493_10159055015087494,Facebook,2020-06-06 20:16:03,2021-09-03 11:28:55,link,Protesters pour into DC for city's largest dem...,fox5dc.com,Protesters pour into DC for city's largest dem...,"Here's the scene in downtown DC, where up to 2...",1,https://www.fox5dc.com/news/protesters-pour-in...,https://www.facebook.com/49727872493/posts/101...,1230384,6.021978,1.0,360,108,421,139,32,11,4,21,0,16,45,27,44,7,16,9,18,16,0,5,48870,Fox 5 DC,fox5dc,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Welcome to the official Facebook page for FOX ...,True,2008-12-16 17:46:26,en,0,48870|10159055015087494,{'DC': 1},{'DC': 1},{'DC': 1}
4,4,50614167032_10158759715837033,Facebook,2020-06-06 20:15:57,2021-08-31 04:12:00,link,Outspoken Hoke County Sheriff Sounds Off on Ra...,spectrumlocalnews.com,Peterkin talks about what needs to be done.,Spectrum News anchor Tim Boyum chats with Hoke...,2,http://specne.ws/oHwpnh,https://www.facebook.com/50614167032/posts/101...,134561,-3.666667,1.0,4,1,0,4,0,0,0,0,0,0,5,3,4,3,2,3,7,6,0,3,400156,Spectrum News 1 North Carolina,SpectrumNews1NC,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Spectrum News 1 is a 24-hour local news channe...,True,2008-10-03 00:36:16,en,0,400156|10158759715837033,"{'Hoke County': 1, 'Raeford': 1}",{},{}
...,...,...,...,...,...,...,...,...,...,...,

In [29]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [30]:
#ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [31]:
###
ct_ner = ct

In [32]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,"{'Washington': 1, 'D.C.': 1, 'Minneapolis': 1}",{},{}
1,{},{},{}
2,{},{},{}
3,{'DC': 1},{'DC': 1},{'DC': 1}
4,"{'Hoke County': 1, 'Raeford': 1}",{},{}


In [33]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

In [34]:
ct_ner['NER-msg']

0        [Washington, Minneapolis, D.C.]
1                                     []
2                                     []
3                                   [DC]
4                 [Hoke County, Raeford]
                      ...               
11360          [Raeford, North Carolina]
11361                                 []
11362                          [Buffalo]
11363                       [Sacramento]
11364           [Florida, Monroe County]
Name: NER-msg, Length: 11365, dtype: object

In [35]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

In [36]:
ct_ner['NER']

0                          [Washington, Minneapolis, D.C.]
1                                                       []
2                                                       []
3                                             [DC, DC, DC]
4                                   [Hoke County, Raeford]
                               ...                        
11360    [Raeford, North Carolina, Raeford, North Carol...
11361                                                   []
11362                 [Buffalo, Buffalo, BUFFALO, Buffalo]
11363              [Sacramento, Buffalo, BUFFALO, Buffalo]
11364     [Florida, Monroe County, Florida, Monroe County]
Name: NER, Length: 11365, dtype: object

In [37]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

In [38]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

In [39]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER
0,0,19729044948_10151472828499949,Facebook,2020-06-06 20:16:26,2021-07-28 15:31:24,photo,NaN,NaN,"Yesterdays webinar Gender-Justice was, hopeful...",Protestors gather along 16th Street NW near th...,1,https://www.facebook.com/933thebeat/photos/a.3...,https://www.facebook.com/19729044948/posts/101...,56103,-6.000000,1.0,3,1,0,0,0,0,0,0,0,0,3,3,2,3,2,4,7,0,0,6,38774,93.3 The Beat Jamz,933thebeat,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Jacksonville's #1 Station for Hip-Hop and R&B ...,True,2009-01-13 16:46:07,en,0,38774|10151472828499949,"[Washington, Minneapolis, D.C.]",[],[],"[washington, minneapolis, d.c.]"
1,1,193425634038780_2897594533621863,Facebook,2020-06-06 20:16:26,2021-06-15 21:27:28,link,George Floyd GoFundMe Campaign Has Received Mo...,incrediblestories.me,A GoFundMe page set up for George Floyd has re...,George Floyd GoFundMe Campaign Has Received Mo...,1,https://incrediblestories.me/george-floyd-gofu...,https://www.facebook.com/193425634038780/posts...,528349,3.857143,1.0,457,100,270,258,20,2,0,0,0,51,146,24,39,54,7,8,5,4,0,8,369803,Terry Bаms,TerryBams,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Army Vet | Best Selling Author | Father | Husb...,True,2011-06-14 05:05:53,en,0,369803|2897594533621863,[],[],[],[]
2,2,52150999700_10158798739704701,Facebook,2020-06-06 20:16:22,2021-09-02 05:24:41,link,Watch 'Sesame Street' and CNN's town hall on r...,ew.com,'Sesame Street' and CNN presented a one-hour s...,Sesame Street and CNN presented a one-hour spe...,1,http://share.ew.com/X8cbJCW,https://www.facebook.com/52150999700/posts/101...,4609669,1.461864,1.0,628,174,32,188,3,6,1,3,0,36,477,35,52,81,15,19,23,6,0,11,15356,Entertainment Weekly,entertainmentweekly,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Entertainment Weekly brings the fun. With our...,True,2009-02-23 20:21:13,en,0,15356|10158798739704701,[],[],[],[]
3,3,49727872493_10159055015087494,Facebook,2020-06-06 20:16:03,2021-09-03 11:28:55,link,Protesters pour into DC for city's largest dem...,fox5dc.com,Protesters pour into DC for city's largest dem...,"Here's the scene in downtown DC, where up to 2...",1,https://www.fox5dc.com/news/protesters-pour-in...,https://www.facebook.com/49727872493/posts/101...,1230384,6.021978,1.0,360,108,421,139,32,11,4,21,0,16,45,27,44,7,16,9,18,16,0,5,48870,Fox 5 DC,fox5dc,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Welcome to the official Facebook page for FOX ...,True,2008-12-16 17:46:26,en,0,48870|10159055015087494,[DC],[DC],[DC],[dc]
4,4,50614167032_10158759715837033,Facebook,2020-06-06 20:15:57,2021-08-31 04:12:00,link,Outspoken Hoke County Sheriff Sounds Off on Ra...,spectrumlocalnews.com,Peterkin talks about what needs to be done.,Spectrum News anchor Tim Boyum chats with Hoke...,2,http://specne.ws/oHwpnh,https://www.facebook.com/50614167032/posts/101...,134561,-3.666667,1.0,4,1,0,4,0,0,0,0,0,0,5,3,4,3,2,3,7,6,0,3,400156,Spectrum News 1 North Carolina,SpectrumNews1NC,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Spectrum News 1 is a 24-hour local news channe...,True,2008-10-03 00:36:16,en,0,400156|10158759715837033,"[Hoke County, Raeford]",[],[],"[hoke county, raeford]"
...,...,..

In [40]:
ct_ner['NER-1']=ct_ner['NER']

In [41]:
ct_ner['NER-1'][0]

['washington', 'minneapolis', 'd.c.']

In [42]:
!pip install -q geopy
!pip install -q pyproj

# basic utilities
import json
import requests
import pandas as pd
import pprint
import time

# reverse geocoding utilities
from geopy.geocoders import Nominatim

# geography projection utilities
import pyproj
from shapely.ops import transform
from shapely.geometry import Point
from functools import partial

     |████████████████████████████████| 6.3 MB 12.7 MB/s 


In [43]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [44]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,19729044948_10151472828499949,Facebook,2020-06-06 20:16:26,2021-07-28 15:31:24,photo,NaN,NaN,"Yesterdays webinar Gender-Justice was, hopeful...",Protestors gather along 16th Street NW near th...,1,https://www.facebook.com/933thebeat/photos/a.3...,https://www.facebook.com/19729044948/posts/101...,56103,-6.000000,1.0,3,1,0,0,0,0,0,0,0,0,3,3,2,3,2,4,7,0,0,6,38774,93.3 The Beat Jamz,933thebeat,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Jacksonville's #1 Station for Hip-Hop and R&B ...,True,2009-01-13 16:46:07,en,0,38774|10151472828499949,"[Washington, Minneapolis, D.C.]",[],[],"[washington, minneapolis, d.c.]","[washington, minneapolis, d.c.]"
1,1,193425634038780_2897594533621863,Facebook,2020-06-06 20:16:26,2021-06-15 21:27:28,link,George Floyd GoFundMe Campaign Has Received Mo...,incrediblestories.me,A GoFundMe page set up for George Floyd has re...,George Floyd GoFundMe Campaign Has Received Mo...,1,https://incrediblestories.me/george-floyd-gofu...,https://www.facebook.com/193425634038780/posts...,528349,3.857143,1.0,457,100,270,258,20,2,0,0,0,51,146,24,39,54,7,8,5,4,0,8,369803,Terry Bаms,TerryBams,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Army Vet | Best Selling Author | Father | Husb...,True,2011-06-14 05:05:53,en,0,369803|2897594533621863,[],[],[],[],[]
2,2,52150999700_10158798739704701,Facebook,2020-06-06 20:16:22,2021-09-02 05:24:41,link,Watch 'Sesame Street' and CNN's town hall on r...,ew.com,'Sesame Street' and CNN presented a one-hour s...,Sesame Street and CNN presented a one-hour spe...,1,http://share.ew.com/X8cbJCW,https://www.facebook.com/52150999700/posts/101...,4609669,1.461864,1.0,628,174,32,188,3,6,1,3,0,36,477,35,52,81,15,19,23,6,0,11,15356,Entertainment Weekly,entertainmentweekly,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Entertainment Weekly brings the fun. With our...,True,2009-02-23 20:21:13,en,0,15356|10158798739704701,[],[],[],[],[]
3,3,49727872493_10159055015087494,Facebook,2020-06-06 20:16:03,2021-09-03 11:28:55,link,Protesters pour into DC for city's largest dem...,fox5dc.com,Protesters pour into DC for city's largest dem...,"Here's the scene in downtown DC, where up to 2...",1,https://www.fox5dc.com/news/protesters-pour-in...,https://www.facebook.com/49727872493/posts/101...,1230384,6.021978,1.0,360,108,421,139,32,11,4,21,0,16,45,27,44,7,16,9,18,16,0,5,48870,Fox 5 DC,fox5dc,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Welcome to the official Facebook page for FOX ...,True,2008-12-16 17:46:26,en,0,48870|10159055015087494,[DC],[DC],[DC],[dc],[dc]
4,4,50614167032_10158759715837033,Facebook,2020-06-06 20:15:57,2021-08-31 04:12:00,link,Outspoken Hoke County Sheriff Sounds Off on Ra...,spectrumlocalnews.com,Peterkin talks about what needs to be done.,Spectrum News anchor Tim Boyum chats with Hoke...,2,http://specne.ws/oHwpnh,https://www.facebook.com/50614167032/posts/101...,134561,-3.666667,1.0,4,1,0,4,0,0,0,0,0,0,5,3,4,3,2,3,7,6,0,3,400156,Spectrum News 1 North Carolina,SpectrumNews1NC,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Spectrum News 1 is a 24-hour local news channe...,True,2008-10-03 00:36:16,en,0,400156|10158759715837033,"[Hoke County, 

In [45]:
%cd mapping_US

/content/gdrive/My Drive/2021-online-language-offline-impact-main/mapping_US


In [46]:
!ls

dataframe0529_post.csv	    dataframe_mapping_0603.csv
dataframe0530_post.csv	    dataframe_mapping_0604.csv
dataframe0531_post.csv	    dataframe_mapping_0605.csv
dataframe0602_post.csv	    dataframe_mapping_0606.csv
dataframe0603_post.csv	    dataframe_mapping_0607.csv
dataframe_mapping_0529.csv  dataframe_mapping_0608.csv
dataframe_mapping_0530.csv  dataframe_mapping_0609.csv
dataframe_mapping_0531.csv  dataframe_mapping_0610.csv
dataframe_mapping_0601.csv  dataframe_mapping_0611.csv
dataframe_mapping_0602.csv


In [47]:
ct_mapping=pd.read_csv('dataframe_mapping_0603.csv',encoding='ISO-8859-1')

In [48]:
len(ct_mapping['City'])

3082

In [49]:
len(ct_ner['NER-1'])

11365

In [50]:
for k in range(len(ct_ner['NER-1'])):
  list=[]
  for i in ct_ner['NER-1'][k]:
    for j in range(len(ct_mapping['City'])):
      if(i==ct_mapping['City'][j]):
        list.append(ct_mapping['Mapping'][j])
  ct_ner['NER-1'][k]=list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [51]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,19729044948_10151472828499949,Facebook,2020-06-06 20:16:26,2021-07-28 15:31:24,photo,NaN,NaN,"Yesterdays webinar Gender-Justice was, hopeful...",Protestors gather along 16th Street NW near th...,1,https://www.facebook.com/933thebeat/photos/a.3...,https://www.facebook.com/19729044948/posts/101...,56103,-6.000000,1.0,3,1,0,0,0,0,0,0,0,0,3,3,2,3,2,4,7,0,0,6,38774,93.3 The Beat Jamz,933thebeat,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Jacksonville's #1 Station for Hip-Hop and R&B ...,True,2009-01-13 16:46:07,en,0,38774|10151472828499949,"[Washington, Minneapolis, D.C.]",[],[],"[washington, minneapolis, d.c.]","[Washington, District of Columbia, United Stat..."
1,1,193425634038780_2897594533621863,Facebook,2020-06-06 20:16:26,2021-06-15 21:27:28,link,George Floyd GoFundMe Campaign Has Received Mo...,incrediblestories.me,A GoFundMe page set up for George Floyd has re...,George Floyd GoFundMe Campaign Has Received Mo...,1,https://incrediblestories.me/george-floyd-gofu...,https://www.facebook.com/193425634038780/posts...,528349,3.857143,1.0,457,100,270,258,20,2,0,0,0,51,146,24,39,54,7,8,5,4,0,8,369803,Terry Bаms,TerryBams,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Army Vet | Best Selling Author | Father | Husb...,True,2011-06-14 05:05:53,en,0,369803|2897594533621863,[],[],[],[],[]
2,2,52150999700_10158798739704701,Facebook,2020-06-06 20:16:22,2021-09-02 05:24:41,link,Watch 'Sesame Street' and CNN's town hall on r...,ew.com,'Sesame Street' and CNN presented a one-hour s...,Sesame Street and CNN presented a one-hour spe...,1,http://share.ew.com/X8cbJCW,https://www.facebook.com/52150999700/posts/101...,4609669,1.461864,1.0,628,174,32,188,3,6,1,3,0,36,477,35,52,81,15,19,23,6,0,11,15356,Entertainment Weekly,entertainmentweekly,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Entertainment Weekly brings the fun. With our...,True,2009-02-23 20:21:13,en,0,15356|10158798739704701,[],[],[],[],[]
3,3,49727872493_10159055015087494,Facebook,2020-06-06 20:16:03,2021-09-03 11:28:55,link,Protesters pour into DC for city's largest dem...,fox5dc.com,Protesters pour into DC for city's largest dem...,"Here's the scene in downtown DC, where up to 2...",1,https://www.fox5dc.com/news/protesters-pour-in...,https://www.facebook.com/49727872493/posts/101...,1230384,6.021978,1.0,360,108,421,139,32,11,4,21,0,16,45,27,44,7,16,9,18,16,0,5,48870,Fox 5 DC,fox5dc,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Welcome to the official Facebook page for FOX ...,True,2008-12-16 17:46:26,en,0,48870|10159055015087494,[DC],[DC],[DC],[dc],"[Washington, District of Columbia, United States]"
4,4,50614167032_10158759715837033,Facebook,2020-06-06 20:15:57,2021-08-31 04:12:00,link,Outspoken Hoke County Sheriff Sounds Off on Ra...,spectrumlocalnews.com,Peterkin talks about what needs to be done.,Spectrum News anchor Tim Boyum chats with Hoke...,2,http://specne.ws/oHwpnh,https://www.facebook.com/50614167032/posts/101...,134561,-3.666667,1.0,4,1,0,4,0,0,0,0,0,0,5,3,4,3,2,3,7,6,0,3,400156,Spectrum News 1 North Carolina,SpectrumNews1NC,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Spectrum News 1 is a 24-hour local news channe...,True,

In [52]:
ct_ner['NER-1'][12]

[]

In [53]:
ct_ner.to_csv('dataframe0606_post.csv')